In [1]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from PIL import ImageGrab
import logging
import random

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import ImageGrab

In [2]:
known_good_images_folder = r"D:\training_images\test\valid"
test_image_path = r'C:\temp\engineer_typing3.png'
siamese_model_path = r'models\lung_ct_siamese_network_weights_with_crop2.h5'

In [3]:
class ModelPredictor:
    # def __init__(self, siamese_model_path, known_good_images_folder, threshold=0.5):
        # self.siamese_model_path = siamese_model_path
        # self.known_good_images_folder = known_good_images_folder
        # self.siamese_model = self.load_siamese_model(siamese_model_path)
        # self.threshold = threshold  # Set a default or configurable threshold for similarity
        # self.known_good_images = self.preload_known_good_images()  # Assuming this method preloads and processes the images
    def __init__(self, siamese_model_path, known_good_images_folder, threshold=0.5, jury_size=12):
        self.siamese_model_path = siamese_model_path
        self.known_good_images_folder = known_good_images_folder
        self.siamese_model = self.load_siamese_model(siamese_model_path)
        self.threshold = threshold
        self.jury_size = jury_size
        self.known_good_images = self.preload_known_good_images()

    def preload_known_good_images(self):
        # Your existing method to preload images
        print("Preloading known good images...")
        image_paths = []
        for root, dirs, files in os.walk(self.known_good_images_folder):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    full_path = os.path.join(root, file)
                    image_paths.append(full_path)
        return image_paths
        # Cache known good images if needed here
        
    # Continue to define ModelPredictor class
    def load_siamese_model(self, siamese_model_path):
        # Define the base network architecture
        def initialize_base_network(input_shape):
            input = Input(shape=input_shape)
            x = Conv2D(64, (3, 3), activation='relu')(input)
            x = MaxPooling2D((2, 2))(x)
            x = Conv2D(128, (3, 3), activation='relu')(x)
            x = MaxPooling2D((2, 2))(x)
            x = Flatten()(x)
            x = Dense(128, activation='relu')(x)
            return Model(input, x)

        # Rebuild the Siamese network architecture
        input_shape = (152, 152, 1)
        base_network = initialize_base_network(input_shape)
        input_a = Input(shape=input_shape)
        input_b = Input(shape=input_shape)
        processed_a = base_network(input_a)
        processed_b = base_network(input_b)
        distance = Lambda(lambda tensors: tf.sqrt(tf.reduce_sum(tf.square(tensors[0] - tensors[1]), axis=1, keepdims=True)))([processed_a, processed_b])
        model = Model([input_a, input_b], distance)
        model.load_weights(siamese_model_path)  # Load the saved weights
        print("Siamese model loaded successfully.")
        return model
    
    def predict_siamese(self, test_image_path):
        test_image_processed = self.preprocess_image(test_image_path)
        if test_image_processed is None:
            print(f"Failed to preprocess test image at {test_image_path}")
            return None

        test_image_processed = np.expand_dims(test_image_processed, axis=0)  # Adjust as necessary

        # Randomly select a subset of known good images to compare against
        selected_good_images = random.sample(self.known_good_images, min(self.jury_size, len(self.known_good_images)))

        votes = []
        for known_good_image_path in selected_good_images:
            known_good_image_processed = self.preprocess_image(known_good_image_path)
            if known_good_image_processed is None:
                continue  # Skip this pair if the known good image can't be processed

            known_good_image_processed = np.expand_dims(known_good_image_processed, axis=0)  # Adjust as necessary

            # Prepare the pair
            image_pair = [test_image_processed, known_good_image_processed]

            # Make prediction
            prediction_distance = self.siamese_model.predict(image_pair)
            is_similar = prediction_distance < self.threshold  # Threshold to determine similarity

            # Debugging output
            print(f"Comparing {test_image_path} with {known_good_image_path}: Distance = {prediction_distance}, Similar = {is_similar}")
            votes.append(is_similar)

        # Calculate the majority vote
        num_similar = sum(votes)
        majority_similar = num_similar > len(votes) / 2
        print(f"Total votes for 'Similar': {num_similar}/{len(votes)}. Final verdict: {'Normal' if majority_similar else 'Anomalous'}")

        return majority_similar

    def capture_screen(self, coordinates):
        screenshot = ImageGrab.grab(bbox=coordinates)
        return screenshot

    def compare_to_known_images(self, captured_image, threshold=0.5, jury_size=3):
        processed_captured_image = self.preprocess_data(captured_image)
        verdicts = []

        for _ in range(jury_size):
            comparison_image = np.random.choice(self.known_good_images)
            prediction = self.siamese_model.predict([processed_captured_image, comparison_image])
            verdicts.append(prediction < threshold)

        # Determine if the majority verdict is 'similar' or 'dissimilar'
        final_verdict = sum(verdicts) >= jury_size / 2
        return final_verdict
    
    def evaluate_image(self, coordinates):
        # Capture the image from screen coordinates
        captured_image = self.capture_screen(coordinates)

        # Compare to known images to get a verdict
        is_normal = self.compare_to_known_images(captured_image)

        return is_normal
    
    def predict_similarity(self, test_image_path):
        test_image_processed = self.preprocess_image(test_image_path)

        if test_image_processed is None:
            print(f"Failed to preprocess test image at {test_image_path}")
            return "Preprocessing Failed"  # Or handle this scenario as you see fit

        test_image_processed = np.expand_dims(test_image_processed, axis=0)  # Add batch dimension

        similarities = []
        for known_good_image_path in self.known_good_images:
            known_good_image_processed = self.preprocess_image(known_good_image_path)

            if known_good_image_processed is None:
                continue  # Skip this known good image or handle as needed

            known_good_image_processed = np.expand_dims(known_good_image_processed, axis=0)  # Add batch dimension

            # Make prediction (assuming your model expects a list of two inputs for the pairs)
            prediction = self.siamese_model.predict([test_image_processed, known_good_image_processed])

            similarities.append(prediction[0])

        # Decide based on predictions
        # Example: consider 'Similar' if any prediction is below a certain threshold
        similarity_threshold = 0.5  # Adjust based on your model and validation results
        is_similar = any(sim < similarity_threshold for sim in similarities)

        return 'Similar' if is_similar else 'Dissimilar'


    def preprocess_image(self, img_path: str, target_size=(152, 152)):
        #print(f"Attempting to open image: {img_path}")
        try:
            image = load_img(img_path, target_size=target_size, color_mode='grayscale')
            image = img_to_array(image)
            image /= 255.0  # Normalize to [0, 1]
        except FileNotFoundError as e:
            print(f"Failed to open image at {img_path}: {e}")
            return None
        return image

In [4]:
predictor = ModelPredictor(siamese_model_path, known_good_images_folder)



Siamese model loaded successfully.
Preloading known good images...


In [5]:
test_image_path = r'C:\temp\engineer_typing3.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 94ms/step
Comparing C:\temp\engineer_typing3.png with D:\training_images\test\valid\zoomed\randomized_wl\61b0a2d5-c859-4ceb-95e5-3cf38b70849f_1.png: Distance = [[1.1099311]], Similar = [[False]]
1/1 [==============================] - 0s 23ms/step
Comparing C:\temp\engineer_typing3.png with D:\training_images\test\valid\randomized_wl\cropped\eb878dca-26ce-4a8a-8565-2088bdaa8d78_0.png: Distance = [[0.66221154]], Similar = [[False]]
1/1 [==============================] - 0s 22ms/step
Comparing C:\temp\engineer_typing3.png with D:\training_images\test\valid\zoomed\randomized_wl\c5423e86-1937-443f-abe0-61de2d28a56c_0.png: Distance = [[1.1099311]], Similar = [[False]]
1/1 [==============================] - 0s 24ms/step
Comparing C:\temp\engineer_typing3.png with D:\training_images\test\valid\dummy_class\58261a16-4987-41e9-8667-0e99af433e51.png: Distance = [[1.1099311]], Similar = [[False]]
1/1 [==============================] - 0s 23ms/step
Comparing

array([[False]])

In [6]:
test_image_path = r'D:\training_images\test\valid\rotate_and_flip\0ac5863d-51b1-460f-abe1-3e330c596585.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 22ms/step
Comparing D:\training_images\test\valid\rotate_and_flip\0ac5863d-51b1-460f-abe1-3e330c596585.png with D:\training_images\test\valid\zoomed\randomized_wl\800e1493-0d6f-4165-84c7-1002313064c1_0.png: Distance = [[0.19207215]], Similar = [[ True]]
1/1 [==============================] - 0s 22ms/step
Comparing D:\training_images\test\valid\rotate_and_flip\0ac5863d-51b1-460f-abe1-3e330c596585.png with D:\training_images\test\valid\rotate_and_flip\acbdd960-9dff-473c-959c-f35b469225b7.png: Distance = [[0.19207215]], Similar = [[ True]]
1/1 [==============================] - 0s 23ms/step
Comparing D:\training_images\test\valid\rotate_and_flip\0ac5863d-51b1-460f-abe1-3e330c596585.png with D:\training_images\test\valid\dummy_class\ff655528-79b0-40bb-9620-e842a752f01c.png: Distance = [[0.19207215]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\training_images\test\valid\rotate_and_flip\0ac5863d-51b1-460f-abe

array([[ True]])

In [7]:
test_image_path = r'D:\Custom_invalid\cat.jpg'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing D:\Custom_invalid\cat.jpg with D:\training_images\test\valid\randomized_wl\cropped\07a9cef1-8ebd-4af0-b1f7-790929436a86_1.png: Distance = [[0.95939994]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\Custom_invalid\cat.jpg with D:\training_images\test\valid\zoomed\randomized_wl\a6d14009-aff5-4f2f-bfa7-22f6a58a4db8_0.png: Distance = [[0.95939994]], Similar = [[False]]
1/1 [==============================] - 0s 24ms/step
Comparing D:\Custom_invalid\cat.jpg with D:\training_images\test\valid\randomized_wl\cropped\ff7c739b-e765-409b-a1f9-cc5f2a9b814e_0.png: Distance = [[0.95939994]], Similar = [[False]]
1/1 [==============================] - 0s 23ms/step
Comparing D:\Custom_invalid\cat.jpg with D:\training_images\test\valid\randomized_wl\cropped\ebb2328f-df45-44e2-a9a1-96a033943045_0.png: Distance = [[0.95939994]], Similar = [[False]]
1/1 [==============================] - 0s 22ms/step
Compa

array([[False]])

In [8]:
test_image_path = r'D:\custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 22ms/step
Comparing D:\custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with D:\training_images\test\valid\rotate_and_flip\8d3af466-644b-49ee-8d07-03178c57a645.png: Distance = [[0.]], Similar = [[ True]]
1/1 [==============================] - 0s 23ms/step
Comparing D:\custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with D:\training_images\test\valid\randomized_wl\cropped\850926d7-3925-4725-90b7-d3fdab616b7c_1.png: Distance = [[0.]], Similar = [[ True]]
1/1 [==============================] - 0s 22ms/step
Comparing D:\custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with D:\training_images\test\valid\randomized_wl\cropped\dd53d8e1-6a1c-445b-b891-070ab691b898_0.png: Distance = [[0.]], Similar = [[ True]]
1/1 [==============================] - 0s 24ms/step
Comparing D:\custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with D:\training_images\test\valid\zoomed\random

array([[ True]])

In [9]:
test_image_path = r'D:\custom_test_valid\istockphoto-493741910-612x612.jpg'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing D:\custom_test_valid\istockphoto-493741910-612x612.jpg with D:\training_images\test\valid\randomized_wl\cropped\181b3513-c653-4e3d-90b2-c5a0bb723b67_0.png: Distance = [[0.24239028]], Similar = [[ True]]
1/1 [==============================] - 0s 20ms/step
Comparing D:\custom_test_valid\istockphoto-493741910-612x612.jpg with D:\training_images\test\valid\dummy_class\5e5c2dd6-630c-450f-9073-d8f7b1c97c35.png: Distance = [[0.27521837]], Similar = [[ True]]
1/1 [==============================] - 0s 22ms/step
Comparing D:\custom_test_valid\istockphoto-493741910-612x612.jpg with D:\training_images\test\valid\dummy_class\ef468eb7-987c-4bd9-9b83-23b82b673775.png: Distance = [[0.27521837]], Similar = [[ True]]
1/1 [==============================] - 0s 23ms/step
Comparing D:\custom_test_valid\istockphoto-493741910-612x612.jpg with D:\training_images\test\valid\rotate_and_flip\6371913b-743a-422e-94c9-3d7d837aa87b.png: Distance = [[0.2752

array([[ True]])

In [10]:
test_image_path = r'D:\custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 20ms/step
Comparing D:\custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with D:\training_images\test\valid\rotate_and_flip\3d4509e0-f20e-45ad-bc02-de6dc70984cc.png: Distance = [[0.4097311]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with D:\training_images\test\valid\randomized_wl\cropped\9cf004c5-e845-481c-a712-129a4432be8b_0.png: Distance = [[0.4097311]], Similar = [[ True]]
1/1 [==============================] - 0s 22ms/step
Comparing D:\custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with D:\training_images\test\valid\randomized_wl\cropped\4db13d59-613f-4f98-b708-451930704fae_1.png: Distance = [[0.4097311]], Similar = [[ True]]
1/1 [==============================] - 0s 23ms/step
Comparing D:\custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with D:\training_images

array([[ True]])

In [11]:
test_image_path = r'D:\custom_invalid\istockphoto-with_arrow.jpg'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 22ms/step
Comparing D:\custom_invalid\istockphoto-with_arrow.jpg with D:\training_images\test\valid\dummy_class\f0a1e6ca-66a9-4356-8745-393d7cd1bfd2.png: Distance = [[1.1128287]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\custom_invalid\istockphoto-with_arrow.jpg with D:\training_images\test\valid\zoomed\randomized_wl\61a2ba4b-c740-48f3-bf27-e135ab6eb87e_1.png: Distance = [[1.1128287]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\custom_invalid\istockphoto-with_arrow.jpg with D:\training_images\test\valid\zoomed\randomized_wl\453d8321-65f0-4d13-9ed0-6090e917e113_1.png: Distance = [[1.1128287]], Similar = [[False]]
1/1 [==============================] - 0s 19ms/step
Comparing D:\custom_invalid\istockphoto-with_arrow.jpg with D:\training_images\test\valid\rotate_and_flip\cf8ce6b7-f9b1-4c52-812c-3a5ba6dbfa19.png: Distance = [[1.1128287]], Similar = [[False]]
1/1 [=

array([[False]])

In [12]:
test_image_path = r'D:\custom_invalid\Lung_abscess_-_CT_with_overlay.jpg'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 22ms/step
Comparing D:\custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with D:\training_images\test\valid\rotate_and_flip\642e1050-3997-477a-a624-0fbcb93e1209.png: Distance = [[0.886097]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with D:\training_images\test\valid\zoomed\randomized_wl\c790c80a-c245-44ee-a5f1-c06666e9b951_1.png: Distance = [[0.886097]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with D:\training_images\test\valid\randomized_wl\cropped\d16c9743-3f0f-4f88-a918-31dd0f569e99_1.png: Distance = [[0.078857]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with D:\training_images\test\valid\randomized_wl\cropped\fab8297a-ff45-496b-8608-c37a7636042e_0.png: Distance =

array([[False]])

In [13]:
test_image_path = r'c:\temp\ml.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing c:\temp\ml.png with D:\training_images\test\valid\zoomed\randomized_wl\a6de63a6-fe37-4a27-8f35-1e101212fc89_1.png: Distance = [[1.0298642]], Similar = [[False]]
1/1 [==============================] - 0s 23ms/step
Comparing c:\temp\ml.png with D:\training_images\test\valid\randomized_wl\cropped\ed43e850-2dec-4b56-b3e6-aba1f234c287_0.png: Distance = [[1.0298642]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing c:\temp\ml.png with D:\training_images\test\valid\randomized_wl\cropped\44374976-c8c3-4761-bdb8-03e5ff85bb87_0.png: Distance = [[1.0298642]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing c:\temp\ml.png with D:\training_images\test\valid\randomized_wl\cropped\1dd0816f-40bd-4934-865e-40919aee2f9d_0.png: Distance = [[1.0298642]], Similar = [[False]]
1/1 [==============================] - 0s 22ms/step
Comparing c:\temp\ml.png with D:\training_images\test

array([[False]])

In [14]:
test_image_path = r'D:\Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing D:\Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with D:\training_images\test\valid\randomized_wl\cropped\a6e9a6df-2152-4c27-b161-9a1607ce4ef9_1.png: Distance = [[0.13289288]], Similar = [[ True]]
1/1 [==============================] - 0s 20ms/step
Comparing D:\Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with D:\training_images\test\valid\rotate_and_flip\e820d13c-6209-4277-8a7f-97a1cd6dbd23.png: Distance = [[0.]], Similar = [[ True]]
1/1 [==============================] - 0s 20ms/step
Comparing D:\Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with D:\training_images\test\valid\randomized_wl\cropped\7ecc04c4-8bc5-4b85-868e-2bf99c079689_0.png: Distance = [[0.]], Similar = [[ True]]
1/1 [==============================] - 0s 23ms/step
Comparing D:\Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with D:\training_images\test\valid\zoomed\randomized_wl\

array([[ True]])

In [15]:
test_image_path = r'C:\temp\medical_image_black_borders.png'
predictor.predict_siamese(test_image_path)

Failed to open image at C:\temp\medical_image_black_borders.png: [Errno 2] No such file or directory: 'C:\\temp\\medical_image_black_borders.png'
Failed to preprocess test image at C:\temp\medical_image_black_borders.png


In [16]:
test_image_path = r'C:\temp\medical_image_no_borders.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\medical_image_no_borders.png with D:\training_images\test\valid\randomized_wl\cropped\c7b7167a-2ec5-4688-a5cf-e0f36c24c66b_1.png: Distance = [[0.78186643]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_no_borders.png with D:\training_images\test\valid\rotate_and_flip\9f40967d-2b37-44d9-9abe-a84ca91b3895.png: Distance = [[0.78186643]], Similar = [[False]]
1/1 [==============================] - 0s 23ms/step
Comparing C:\temp\medical_image_no_borders.png with D:\training_images\test\valid\randomized_wl\cropped\f491fabe-538b-470e-9218-ee084b603fce_0.png: Distance = [[0.78186643]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_no_borders.png with D:\training_images\test\valid\randomized_wl\cropped\29e1358e-52c5-4abe-a66a-bd853e6d538f_0.png: Distance = [[0.78186643]], Similar = [[False]]
1/1 [===============

array([[False]])

In [17]:
test_image_path = r'C:\temp\white_square_black_borders.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 22ms/step
Comparing C:\temp\white_square_black_borders.png with D:\training_images\test\valid\zoomed\randomized_wl\b197975b-6d07-40df-8049-e51791561efd_1.png: Distance = [[0.88069415]], Similar = [[False]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\white_square_black_borders.png with D:\training_images\test\valid\zoomed\randomized_wl\e9406de0-2d76-44a3-91bd-195efd0badd6_0.png: Distance = [[0.88069415]], Similar = [[False]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\white_square_black_borders.png with D:\training_images\test\valid\randomized_wl\cropped\ba003dd0-508d-433d-8736-9528c1ac070d_1.png: Distance = [[0.76999956]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\white_square_black_borders.png with D:\training_images\test\valid\zoomed\randomized_wl\b2df743e-0f8d-4fe0-8fd9-80500a138c5c_0.png: Distance = [[0.88069415]], Similar = [[False]]
1/1 [==

array([[False]])

In [18]:
test_image_path = r'C:\temp\white_square2_black_borders.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\white_square2_black_borders.png with D:\training_images\test\valid\randomized_wl\cropped\1fd6cf8a-0da9-4e25-b39a-e084b6ddc12e_0.png: Distance = [[0.5400592]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\white_square2_black_borders.png with D:\training_images\test\valid\randomized_wl\cropped\fa32afd7-ab3d-45e2-a9bb-bd6dc1073cda_1.png: Distance = [[0.5400592]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\white_square2_black_borders.png with D:\training_images\test\valid\rotate_and_flip\d5265095-4b7f-4bfd-af0e-3ee60951ba90.png: Distance = [[0.5400592]], Similar = [[False]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\white_square2_black_borders.png with D:\training_images\test\valid\rotate_and_flip\d530fb05-7278-4b07-b5ca-9bdde19b239f.png: Distance = [[0.5400592]], Similar = [[False]]
1/1 [===============

array([[False]])

In [19]:
test_image_path = r'C:\temp\white_square3_black_borders.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\white_square3_black_borders.png with D:\training_images\test\valid\randomized_wl\cropped\9e77a6c4-8d53-44ed-a0df-7a8af7a5587f_1.png: Distance = [[0.3412102]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\white_square3_black_borders.png with D:\training_images\test\valid\dummy_class\c4004cfd-53b5-4699-a010-54e64faa9934.png: Distance = [[0.3412102]], Similar = [[ True]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\white_square3_black_borders.png with D:\training_images\test\valid\rotate_and_flip\fab8297a-ff45-496b-8608-c37a7636042e.png: Distance = [[0.31177735]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\white_square3_black_borders.png with D:\training_images\test\valid\randomized_wl\cropped\6266553a-235c-4e19-aeaf-d5212f5cee08_0.png: Distance = [[0.3412102]], Similar = [[ True]]
1/1 [==================

array([[ True]])

In [20]:
test_image_path = r'C:\temp\white_square2.5_black_borders.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\white_square2.5_black_borders.png with D:\training_images\test\valid\zoomed\randomized_wl\538498ef-8f41-4807-83b9-bced7f9d18cb_0.png: Distance = [[0.5299428]], Similar = [[False]]
1/1 [==============================] - 0s 19ms/step
Comparing C:\temp\white_square2.5_black_borders.png with D:\training_images\test\valid\randomized_wl\cropped\81d9eaf5-8f98-4dff-94e2-e03f4eb8a3df_0.png: Distance = [[0.5299428]], Similar = [[False]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\white_square2.5_black_borders.png with D:\training_images\test\valid\zoomed\randomized_wl\4ea4c58a-7645-402d-9839-defa5d3f627b_0.png: Distance = [[0.5299428]], Similar = [[False]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\white_square2.5_black_borders.png with D:\training_images\test\valid\dummy_class\0825e22d-4170-403e-9632-daca06b5313a.png: Distance = [[0.5299428]], Similar = [[False]]
1/1 [=====

array([[False]])

In [21]:
test_image_path = r'C:\temp\medical_image_zoomed.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 23ms/step
Comparing C:\temp\medical_image_zoomed.png with D:\training_images\test\valid\zoomed\randomized_wl\a01b5d19-26b8-4d5d-a9f7-24944ff7cc59_1.png: Distance = [[0.17103626]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed.png with D:\training_images\test\valid\randomized_wl\cropped\422afcef-fe48-4112-9eb8-5393f4b612e5_0.png: Distance = [[0.17103626]], Similar = [[ True]]
1/1 [==============================] - 0s 22ms/step
Comparing C:\temp\medical_image_zoomed.png with D:\training_images\test\valid\randomized_wl\cropped\bed214af-0f8f-460d-8e64-6ea8dc6c2ac5_1.png: Distance = [[0.17103626]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed.png with D:\training_images\test\valid\randomized_wl\cropped\9041adb7-47b1-45df-8836-52a3efb3163e_1.png: Distance = [[0.17103626]], Similar = [[ True]]
1/1 [========================

array([[ True]])

In [22]:
test_image_path = r'C:\temp\medical_image_zoomed_resized_modified_aspect_ratio.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_resized_modified_aspect_ratio.png with D:\training_images\test\valid\zoomed\randomized_wl\e7cbcc27-3993-48fd-99ef-68be6f363cbd_0.png: Distance = [[0.1612988]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_resized_modified_aspect_ratio.png with D:\training_images\test\valid\rotate_and_flip\99774bc4-7597-451f-a0f5-d065231ebd86.png: Distance = [[0.1612988]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_resized_modified_aspect_ratio.png with D:\training_images\test\valid\zoomed\randomized_wl\74c4d0e5-6a6a-4ca6-8262-0e8c4f41af18_1.png: Distance = [[0.1612988]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_resized_modified_aspect_ratio.png with D:\training_images\test\valid\dummy_class\2dc9cf4f-a965-4499-b45

array([[ True]])

In [23]:
test_image_path = r'C:\temp\medical_image_zoomed_more_and_panned_resized_modified_aspect_ratio.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_and_panned_resized_modified_aspect_ratio.png with D:\training_images\test\valid\dummy_class\c4004cfd-53b5-4699-a010-54e64faa9934.png: Distance = [[0.675523]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_and_panned_resized_modified_aspect_ratio.png with D:\training_images\test\valid\rotate_and_flip\f3fd26ed-9ca6-4f18-96c6-249f4264c5bc.png: Distance = [[0.675523]], Similar = [[False]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\medical_image_zoomed_more_and_panned_resized_modified_aspect_ratio.png with D:\training_images\test\valid\zoomed\randomized_wl\b23dc9b0-581f-4abd-8c35-158df0c7040b_1.png: Distance = [[0.675523]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_and_panned_resized_modified_aspect_ratio.png with D:\training_im

array([[False]])

In [24]:
test_image_path = r'C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio.png with D:\training_images\test\valid\zoomed\randomized_wl\70093be2-6889-4e9d-83f9-5271094afbaa_0.png: Distance = [[0.28508013]], Similar = [[ True]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio.png with D:\training_images\test\valid\zoomed\randomized_wl\c79a2de8-2b58-4d20-866d-00fa63dc8efc_0.png: Distance = [[0.28508013]], Similar = [[ True]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio.png with D:\training_images\test\valid\rotate_and_flip\0c3c862c-0dfb-4465-8653-983868e18707.png: Distance = [[0.28508013]], Similar = [[ True]]
1/1 [==============================] - 0s 19ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio.png with D:\training_images\test\valid\zoomed\rand

array([[ True]])

In [25]:
test_image_path = r'c:\temp\medical_image_zoomed_more_rotated_resized_modified_aspect_ratio.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing c:\temp\medical_image_zoomed_more_rotated_resized_modified_aspect_ratio.png with D:\training_images\test\valid\randomized_wl\cropped\d118fc3f-54a8-4883-95e4-688acf20c03d_1.png: Distance = [[0.81463313]], Similar = [[False]]
1/1 [==============================] - 0s 20ms/step
Comparing c:\temp\medical_image_zoomed_more_rotated_resized_modified_aspect_ratio.png with D:\training_images\test\valid\rotate_and_flip\cf95d811-87ae-49db-a9da-6a26a5d33926.png: Distance = [[0.81463313]], Similar = [[False]]
1/1 [==============================] - 0s 20ms/step
Comparing c:\temp\medical_image_zoomed_more_rotated_resized_modified_aspect_ratio.png with D:\training_images\test\valid\randomized_wl\cropped\04ed8ca6-4916-479c-b35d-28b0cfb7d496_1.png: Distance = [[0.81463313]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing c:\temp\medical_image_zoomed_more_rotated_resized_modified_aspect_ratio.png with D:\trai

array([[False]])

In [26]:
test_image_path = r'C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with D:\training_images\test\valid\dummy_class\5801bfcf-bced-4872-89f0-9fdc6012871a.png: Distance = [[0.34389877]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with D:\training_images\test\valid\zoomed\randomized_wl\2cbc4da9-82a2-4c34-a512-433d8bb6c1d8_1.png: Distance = [[0.34389877]], Similar = [[ True]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with D:\training_images\test\valid\randomized_wl\cropped\4387ff33-d58b-4d9d-bbda-09a130026dc0_1.png: Distance = [[0.34389877]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with D:\t

array([[ True]])

In [27]:
test_image_path = r'C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_small_square.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_small_square.png with D:\training_images\test\valid\zoomed\randomized_wl\291dfb02-dedc-46f2-9b1a-0d5fb71970a0_1.png: Distance = [[0.2528959]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_small_square.png with D:\training_images\test\valid\dummy_class\433a1b86-a8e4-47da-9dcf-098e5bbf9cee.png: Distance = [[0.2528959]], Similar = [[ True]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_small_square.png with D:\training_images\test\valid\zoomed\randomized_wl\e63ecb0c-e124-4344-9857-a1a083bf1592_1.png: Distance = [[0.2528959]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_small_square.png w

array([[ True]])

In [28]:
test_image_path = r'C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_medium_square.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_medium_square.png with D:\training_images\test\valid\randomized_wl\cropped\c7e54aee-defe-4481-a2b6-896fc8b65564_0.png: Distance = [[0.2692656]], Similar = [[ True]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_medium_square.png with D:\training_images\test\valid\zoomed\randomized_wl\a5708c2e-5882-4a11-9c3b-438f6b37cc79_1.png: Distance = [[0.2692656]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_medium_square.png with D:\training_images\test\valid\zoomed\randomized_wl\8219f071-0e35-467c-a9e7-ba8f4b5e958f_0.png: Distance = [[0.2692656]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_med

array([[ True]])

In [29]:
test_image_path = r'C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_mediumlarge_square.png'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_mediumlarge_square.png with D:\training_images\test\valid\rotate_and_flip\0bbc2e2d-52fe-47c2-97a0-7b4f5a62680b.png: Distance = [[0.57698804]], Similar = [[False]]
1/1 [==============================] - 0s 21ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_mediumlarge_square.png with D:\training_images\test\valid\dummy_class\ad82138d-1137-4831-a176-24a5df99d727.png: Distance = [[0.57698804]], Similar = [[False]]
1/1 [==============================] - 0s 20ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_mediumlarge_square.png with D:\training_images\test\valid\rotate_and_flip\16fca03c-3082-4e0f-a782-db838966195c.png: Distance = [[0.57698804]], Similar = [[False]]
1/1 [==============================] - 0s 22ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_mediumlarge

array([[False]])

In [30]:
test_image_path = r'D:\Custom_test_valid\internet-gettyimages-1320918955-612x612.jpg'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 20ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1320918955-612x612.jpg with D:\training_images\test\valid\randomized_wl\cropped\68ca8240-9d01-490a-a83c-6d8a4041c598_0.png: Distance = [[0.45216542]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1320918955-612x612.jpg with D:\training_images\test\valid\zoomed\randomized_wl\c0a276e3-ac4a-48c3-b623-f5c8784caa78_0.png: Distance = [[0.]], Similar = [[ True]]
1/1 [==============================] - 0s 20ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1320918955-612x612.jpg with D:\training_images\test\valid\randomized_wl\cropped\1c29c60c-9a16-42a9-9ebd-39dd7cc4bd72_1.png: Distance = [[0.01941449]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1320918955-612x612.jpg with D:\training_images\test\valid\rotate_and_flip\446517

array([[ True]])

In [31]:
test_image_path = r'D:\Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg'
predictor.predict_siamese(test_image_path)

1/1 [==============================] - 0s 21ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with D:\training_images\test\valid\randomized_wl\cropped\eedc8712-4c51-4dc7-9b90-a8a55f4df385_1.png: Distance = [[0.]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with D:\training_images\test\valid\zoomed\randomized_wl\6f2f4d43-c7bf-4434-a8bb-269cd579f873_0.png: Distance = [[0.]], Similar = [[ True]]
1/1 [==============================] - 0s 21ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with D:\training_images\test\valid\zoomed\randomized_wl\68e72bde-439f-4b9d-81ef-423d45f5b736_0.png: Distance = [[0.]], Similar = [[ True]]
1/1 [==============================] - 0s 22ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with D:\training_images\test\valid\rotate_and_flip\9f39dd9a-cebf-4805-b232

array([[ True]])